In [1]:
import json
import logging
import time
from pathlib import Path

import ffmpeg
import numpy as np
import torch
import whisper
import whisper.utils
import classes

source_folder = Path('video_input')
output_folder = Path('subs_out')
model = 'tiny'
language = 'french'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#####################################################################

print('Current configuration:')
print(f'Source folder: {source_folder}')
print(f'Output folder: {output_folder}')
print(f'Model: {model}')
print(f'Language: {language}')
print(f'Device: {device}')


Current configuration:
Source folder: video_input
Output folder: subs_out
Model: medium
Language: french
Device: cuda
Current configuration:
Source folder: video_input
Output folder: subs_out
Model: tiny
Language: french
Device: cuda


In [ ]:
whisper.available_models()

In [2]:
classes.setup_logging()

logging.info("This is an info message.")
logging.error("This is an error message.")

# Init Whisperer
whisperer = classes.Whisperer(output_folder=output_folder, model_name=model, language=language, device=device)
whisperer.init_model()

2023-10-01 15:11:38,700 - root - INFO - This is an info message.
2023-10-01 15:11:38,701 - root - ERROR - This is an error message.
2023-10-01 15:11:38,703 - root - INFO - Loading model tiny on cuda
2023-10-01 15:11:41,485 - root - INFO - Loaded model tiny on cuda


In [3]:
# get list of videos in source folder, extension .mp4 mkv
videos = [f for f in source_folder.iterdir() if f.is_file() and f.suffix in ['.mp4', '.mkv']]
logging.info(f"Found {len(videos)} videos to process")

# read context from file
context_file = Path('context.txt')
context = context_file.read_text() if context_file.exists() else None

2023-10-01 15:11:41,496 - root - INFO - Found 1 videos to process


In [4]:
# Process videos
for video in videos:
    tic = time.time()
    logging.info(f"Processing video {video.name}")
    video_obj = classes.Video(video, save_raw=True)
    video_obj.context = context
    video_obj.generate_subtitles(whisperer)
    logging.info(f"Processed video {video.name} in {time.time() - tic:.1f} sec")

2023-10-01 15:11:41,509 - root - INFO - Processing video 2023-04-28 15-41-25_edit.mp4
2023-10-01 15:11:41,510 - root - INFO - Generating subtitles for video_input\2023-04-28 15-41-25_edit.mp4
2023-10-01 15:11:41,512 - root - INFO - Loading audio from video_input\2023-04-28 15-41-25_edit.mp4
2023-10-01 15:11:41,747 - root - INFO - Loaded audio from video_input\2023-04-28 15-41-25_edit.mp4
c:\Users\winsid\mambaforge\envs\pytrans\Lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\winsid\mambaforge\envs\pytrans\Lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


[00:00.740 --> 00:04.160]  Je comprends pas, il est pas plus certaine annuel, qu'est ce qui s'est passé.
[00:08.040 --> 00:08.800]  On relâche l'intérêt.
[00:10.500 --> 00:14.800]  Il est passant annuel, on va venir pour vous le repasser en auto.
[00:17.200 --> 00:19.580]  Et il faut qu'on est étudié, qu'est ce qui s'est passé avant le repasser en auto.
[00:20.100 --> 00:21.240]  On peut forcément le rentrer en auto.
[00:23.480 --> 00:25.080]  Et tu couches de l'un bonan, je vous relâche.


c:\Users\winsid\mambaforge\envs\pytrans\Lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\winsid\mambaforge\envs\pytrans\Lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


[00:25.080 --> 00:37.760]  Je vois justement d'humanement c'estでは.
[00:41.700 --> 00:44.620]  Je tenais très.
[00:45.760 --> 00:46.180]  Je l'mise.
[00:46.560 --> 00:51.840]  J'l'l'en avais main suis tout le monde.
[00:51.840 --> 00:53.980]  J'en ai mal pour勇 voler beaucoup de SUN Paulo.
[00:53.980 --> 00:55.000]  Je fuge le
[00:55.000 --> 01:00.420]  compte que quand on fera un autre point après pour les modifs collectionnera de monde
[01:00.420 --> 01:02.060]  et sur le monde.
[01:02.160 --> 01:05.500]  Ou ce qu'on a discuté ensemble, ça marche.
[01:06.820 --> 01:08.080]  Merci à vous tous.
[01:08.500 --> 01:10.320]  Bon oui, c'est bon à tous.
[01:16.740 --> 01:19.580]  Le coup, la va être surroulage, des effectivement, des passants magnuels, quoi.
[01:19.920 --> 01:22.300]  Et j'aimerais, et on reviendra comprendre pourquoi.


2023-10-01 15:11:57,547 - root - INFO - Done transcription in 15.8 sec
2023-10-01 15:11:57,548 - root - INFO - Saving raw output for video_input\2023-04-28 15-41-25_edit.mp4
2023-10-01 15:11:57,562 - root - INFO - Saved raw output for video_input\2023-04-28 15-41-25_edit.mp4
2023-10-01 15:11:57,563 - root - INFO - Saving subtitles for video_input\2023-04-28 15-41-25_edit.mp4
2023-10-01 15:11:57,564 - root - INFO - Saving subtitles for video_input\2023-04-28 15-41-25_edit.mp4 to 2023-04-28 15-41-25_edit.srt
2023-10-01 15:11:57,571 - root - INFO - Generated subtitles for video_input\2023-04-28 15-41-25_edit.mp4
2023-10-01 15:11:57,572 - root - INFO - Processed video 2023-04-28 15-41-25_edit.mp4 in 16.1 sec


[01:29.720 --> 01:32.240]  Donc on n'a pas de traiter d'incident pour avoir fonds ensemble sur ça.
[01:34.140 --> 01:37.480]  Mais il y a tout qu'il y a été présenté pour que vous pîiez comme les débeguer.
[01:39.060 --> 01:40.220]  Et pendant ce type d'incident.


In [ ]:
video_obj.raw_output_path / f'{video_obj.path.stem}.mp4'